In [1]:
from pathlib import Path

from sklearn.metrics import f1_score, accuracy_score

from bella import helper
from bella.models.tdlstm import LSTM, TDLSTM, TCLSTM
from bella.models.target import TargetDep
from bella.parsers import semeval_14

Using TensorFlow backend.


# How to download and use the models

THis notebook gives a brief example of how to download the pre-trained model and how to apply it to your own data. We will also show how to apply them to the SemEval dataset.

In [2]:
# Download the model that you want
lstm = helper.download_model(LSTM, 'SemEval 14 Restaurant')
tdlstm = helper.download_model(TDLSTM, 'SemEval 14 Restaurant')
tclstm = helper.download_model(TCLSTM, 'SemEval 14 Restaurant')
target_dep = helper.download_model(TargetDep, 'SemEval 14 Restaurant')
models = [lstm, tdlstm, tclstm, target_dep]

In [3]:
test_example_pos = [{'text' : 'This bread is tasty', 'target': 'bread', 
                     'spans': [(5, 10)]}]
test_example_neg = [{'text' : 'This bread is burnt', 'target': 'bread', 
                     'spans': [(5, 10)]}]
test_example_multi = [{'text' : 'This bread is tasty but the sauce is too rich', 'target': 'sauce', 
                     'spans': [(28, 33)]}]

In [4]:
sentiment_mapper = {0: -1, 1: 0, 2: 1}
for model in models:
    pos_pred = model.predict(test_example_pos)[0]
    neg_pred = model.predict(test_example_neg)[0]
    multi_pred = model.predict(test_example_multi)[0]
    if 'LSTM' in model.name():
        pos_pred = sentiment_mapper[pos_pred]
        neg_pred = sentiment_mapper[neg_pred]
        multi_pred = sentiment_mapper[multi_pred]
    print(f'Model: {model.name()}\n\t Positive correct: {pos_pred==1}\n\t'
          f' Negative correct: {neg_pred==-1}\n\t Multi correct: {multi_pred==-1}\n')

Model: LSTM
	 Positive correct: True
	 Negative correct: False
	 Multi correct: False

Model: TDLSTM
	 Positive correct: True
	 Negative correct: False
	 Multi correct: False

Model: TCLSTM
	 Positive correct: True
	 Negative correct: False
	 Multi correct: False

Model: Target Dependent
	 Positive correct: True
	 Negative correct: True
	 Multi correct: True



As shown above the LSTM models require a sentiment mapper to change the values to the -1, 0, and 1 scaling rather than 0, 1, 2.

## How to apply them to a dataset

We are going to apply the methods to the SemEval 2014 Resturant Test dataset that can be found [here](http://metashare.ilsp.gr:8080/repository/browse/semeval-2014-absa-test-data-gold-annotations/b98d11cec18211e38229842b2b6a04d77591d40acd7542b7af823a54fb03a155/). You do need to download the dataset to do this.

In [5]:
# Path to the SemEVal 2014 Resturant test data set
test_data_path = Path('..', '..', 'ABSA_Gold_TestData', 'Restaurants_Test_Gold.xml')
# Load the dataset
test_data = semeval_14(test_data_path)

In [6]:
sentiment_mapper = {0: -1, 1: 0, 2: 1}
for model in models:
    predictions = model.predict(test_data.data_dict())
    if 'LSTM' in model.name():
        predictions = [sentiment_mapper[pred] for pred in predictions]
    y_test = test_data.sentiment_data()
    f1 = f1_score(y_test, predictions, average='macro') * 100
    acc = accuracy_score(y_test, predictions) * 100
    print(f'Model: {model.name()}, Macro F1: {f1:.2f} '
          f'Accuracy: {acc:.2f}\n')

/home/andrew/Envs/Bella/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Model: LSTM, Macro F1: 46.36 Accuracy: 71.07

Model: TDLSTM, Macro F1: 57.68 Accuracy: 74.38

Model: TCLSTM, Macro F1: 55.38 Accuracy: 72.41

Model: Target Dependent, Macro F1: 56.16 Accuracy: 73.84

